# Logistic regression


#Import

In [1]:
!pip install contractions
import pandas as pd
import contractions
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import nltk.tokenize as nt
import numpy as np

     |████████████████████████████████| 245kB 11.1MB/s 
     |████████████████████████████████| 317kB 23.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=eb984ed0131627848ffd46b065ba08f9c1a8f00a660e58334df898089e4cf38b
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#Prepare Data

In [2]:
!gdown --id 1TvPCXiMeBnvsqdUYUzBhzHqok_W5JQwr

Downloading...
From: https://drive.google.com/uc?id=1TvPCXiMeBnvsqdUYUzBhzHqok_W5JQwr
To: /content/lab4_train.csv
100% 331k/331k [00:00<00:00, 49.0MB/s]


In [3]:
data = pd.read_csv('lab4_train.csv')

In [4]:
def tokenize_filter(string_text):
  filter_words = []
  contracts = contractions.fix(string_text) #เปลี่ยนคำย่อ สมมติว่า didn't เป็น did not 
  ss=nt.sent_tokenize(contracts)
  tokenized_sent=[nt.word_tokenize(sent) for sent in ss]
  pos_sentences=[nltk.pos_tag(sent) for sent in tokenized_sent]
  for lst in pos_sentences:
    for tup in lst:
      if tup[1].startswith(('J','N','R')) and tup[0]:
        filter_words.append(tup[0])
  return filter_words

def contract(string_text):
  contracts = contractions.fix(string_text)
  return contracts.split()

data['tokens'] = data['text'].apply(contract)
data['tokens']

0           [But, the, staff, was, so, horrible, to, us.]
1       [To, be, completely, fair,, the, only, redeemi...
2       [To, be, completely, fair,, the, only, redeemi...
3       [The, food, is, uniformly, exceptional,, with,...
4       [Where, Gabriela, personaly, greets, you, and,...
                              ...                        
3151    [I, would, highly, recommend, this, place, to,...
3152                       [The, service, is, fantastic.]
3153    [I, recommend, that, you, try, this, hidden, s...
3154    [The, garlic, mashed, potatoes, are, hands, do...
3155                     [Food, and, service, was, okay.]
Name: tokens, Length: 3156, dtype: object

In [5]:
data['aspectCategory'].value_counts()

food                       1051
anecdotes/miscellaneous     956
service                     506
ambience                    368
price                       275
Name: aspectCategory, dtype: int64

In [6]:
data ['polarity'].value_counts()

positive    1876
negative     715
neutral      398
conflict     167
Name: polarity, dtype: int64

In [7]:
data

,id,text,aspectCategory,polarity,tokens
0,3121,But the staff was so horrible to us.,service,negative,"[But, the, staff, was, so, horrible, to, us.]"
1,2777,"To be completely fair, the only redeeming fact...",food,positive,"[To, be, completely, fair,, the, only, redeemi..."
2,2777,"To be completely fair, the only redeeming fact...",anecdotes/miscellaneous,negative,"[To, be, completely, fair,, the, only, redeemi..."
3,1634,"The food is uniformly exceptional, with a very...",food,positive,"[The, food, is, uniformly, exceptional,, with,..."
4,2534,Where Gabriela personaly greets you and recomm...,service,positive,"[Where, Gabriela, personaly, greets, you, and,..."
...,...,...,...,...,...
3151,2853,I would highly recommend this place to everyon...,anecdotes/miscellaneous,positive,"[I, would, highly, recommend, this, place, to,..."
3152,1163,The service is fantastic.,service,positive,"[The, service, is, fantastic.]"
3153,216,I recommend that you try this hidden spot whic...,anecdotes/miscellaneous,positive,"[I, recommend, that, you, try, this, hidden, s..."
3154,1109,The garlic mashed potatoes are hands down the ...,food,positive,"[The, garlic, mashed, potatoes, are, hands, do..."


In [8]:
data['tokens'][700]

['Others',
 'have',
 'to',
 'go',
 'to',
 'other',
 'restaurants',
 'and',
 'feel',
 'sad',
 'until',
 'they',
 'are',
 'eaten.']

In [9]:
train, dev = np.split(data, [int(0.8 * len(data))])

#Featurize

In [10]:
def featurize(token_list):
    dic = {}
    for i in range(len(token_list)):
        dic[token_list[i]] = 1
    return dic

def unibi_gram(token_list):
  unibi_dic = {}
  for i in range(len(token_list)-1):
    unibi_dic[token_list[i]] = 1
    unibi_dic[token_list[i] + '_' +token_list[i+1]] = 1
  return unibi_dic

In [11]:
feature_dict_list = [unibi_gram(x) for x in train['tokens']]

In [12]:
feature_dict_list[1]

{'To': 1,
 'To_be': 1,
 'above': 1,
 'above_average,': 1,
 'all': 1,
 'all_the': 1,
 'average,': 1,
 'average,_but': 1,
 'be': 1,
 'be_completely': 1,
 'but': 1,
 'but_could': 1,
 'completely': 1,
 'completely_fair,': 1,
 'could': 1,
 'could_not': 1,
 'deficiencies': 1,
 'deficiencies_of': 1,
 'factor': 1,
 'factor_was': 1,
 'fair,': 1,
 'fair,_the': 1,
 'food,': 1,
 'food,_which': 1,
 'for': 1,
 'for_all': 1,
 'make': 1,
 'make_up': 1,
 'not': 1,
 'not_make': 1,
 'of': 1,
 'of_Teodora.': 1,
 'only': 1,
 'only_redeeming': 1,
 'other': 1,
 'other_deficiencies': 1,
 'redeeming': 1,
 'redeeming_factor': 1,
 'the': 1,
 'the_food,': 1,
 'the_only': 1,
 'the_other': 1,
 'up': 1,
 'up_for': 1,
 'was': 1,
 'was_above': 1,
 'was_the': 1,
 'which': 1,
 'which_was': 1}

#Sentiment analysis

In [13]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer(sparse=True)
train_features = train['tokens'].apply(featurize)
feature_vectors = vectorizer.fit_transform(train_features) 

lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['polarity']) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
dev_featurized_list_dicts = dev['tokens'].apply(featurize)
dev_feature_vector = vectorizer.transform(dev_featurized_list_dicts) 
dev_feature_vector

<632x5686 sparse matrix of type '<class 'numpy.float64'>'
	with 7659 stored elements in Compressed Sparse Row format>

In [15]:
polarity_predictions = lr_text_classifier.predict(dev_feature_vector)
print (classification_report(dev['polarity'], polarity_predictions))

              precision    recall  f1-score   support

    conflict       0.26      0.36      0.30        25
    negative       0.51      0.27      0.35       150
     neutral       0.47      0.21      0.29        77
    positive       0.70      0.89      0.79       380

    accuracy                           0.64       632
   macro avg       0.48      0.43      0.43       632
weighted avg       0.61      0.64      0.60       632



#Aspect analysis

In [16]:
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['aspectCategory']) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
dev_featurized_list_dicts = dev['tokens'].apply(featurize)
dev_feature_vector = vectorizer.transform(dev_featurized_list_dicts) 
dev_feature_vector

<632x5686 sparse matrix of type '<class 'numpy.float64'>'
	with 7659 stored elements in Compressed Sparse Row format>

In [18]:
aspect_predictions = lr_text_classifier.predict(dev_feature_vector)
print (classification_report(dev['aspectCategory'], aspect_predictions))

                         precision    recall  f1-score   support

               ambience       0.54      0.28      0.37        71
anecdotes/miscellaneous       0.68      0.79      0.73       194
                   food       0.62      0.78      0.69       203
                  price       0.42      0.13      0.20        60
                service       0.62      0.56      0.59       104

               accuracy                           0.63       632
              macro avg       0.58      0.51      0.52       632
           weighted avg       0.61      0.63      0.60       632



In [19]:
file = pd.DataFrame()
file['id'] = dev['id']
file['polarity'] = polarity_predictions
file['aspectCategory'] = aspect_predictions
file.to_csv('pred.csv', index=None)
file

,id,polarity,aspectCategory
2524,3235,positive,service
2525,3480,neutral,anecdotes/miscellaneous
2526,1459,neutral,food
2527,1459,neutral,food
2528,1597,positive,price
...,...,...,...
3151,2853,positive,anecdotes/miscellaneous
3152,1163,positive,service
3153,216,positive,anecdotes/miscellaneous
3154,1109,positive,food


In [20]:
!gdown --id 1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_

Downloading...
From: https://drive.google.com/uc?id=1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_
To: /content/evaluate.py
100% 7.03k/7.03k [00:00<00:00, 14.3MB/s]


In [21]:
!python3 evaluate.py lab4_train.csv pred.csv

=== CLASSIFICATION : ASPECT ===
                class name  precision  recall  F1-score support
0                     food      0.849   0.778     0.812     203
1                    price      0.727   0.133     0.225      60
2                  service      0.866   0.558     0.678     104
3                 ambience      0.769   0.282     0.412      71
4  anecdotes/miscellaneous      0.744   0.794     0.768     194
5                MACRO AVG      0.791   0.509     0.579     632
6                MICRO AVG      0.801   0.630     0.705     632 

=== CLASSIFICATION : SENTIMENT ===
  class name  precision  recall  F1-score support
0   positive      0.720   0.892     0.797     306
1   negative      0.522   0.280     0.365     125
2    neutral      0.533   0.216     0.308      74
3   conflict      0.429   0.375     0.400      24
4  MACRO AVG      0.551   0.441     0.467     529
5  MICRO AVG      0.670   0.629     0.649     529 

=== CLASSIFICATION : OVERALL ===
              precision  recall  F